In [ ]:
!pip install Flask pyngrok

In [ ]:
!ngrok authtoken 2mswHotIVN6PA0V0Rbcx7iWEK3P_5VMbF6WHctC4YQgmtW6Ri

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 36.5/107.7 GB disk)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from flask import Flask, request, jsonify, send_from_directory, render_template_string
import subprocess
import os
import cv2
import numpy as np
from pyngrok import ngrok

app = Flask(__name__)

# Directories to save uploaded images and output images
UPLOAD_FOLDER = '/content/drive/MyDrive/internship/uploads/'
OUTPUT_FOLDER_BASE = '/content/drive/MyDrive/internship/predict/'  # Base folder for output
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER_BASE, exist_ok=True)

# Utility functions for image processing
def bluish_grey_to_black(image_path, output_path):
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_blue = np.array([0, 0, 50])
    upper_blue = np.array([180, 50, 200])
    mask = cv2.inRange(hsv_image, lower_blue, upper_blue)
    image[mask > 0] = [0, 0, 0]
    cv2.imwrite(output_path, image)
    return image

def calculate_leaf_area(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    total_area = 256 * 256
    black_area = np.sum(gray_image == 0)
    leaf_area = total_area - black_area
    return leaf_area

def read_yolo_instance_segmentation_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    annotations = []
    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        points = [(float(parts[i]) * 256, float(parts[i + 1]) * 256) for i in range(1, len(parts), 2)]
        annotations.append((class_id, points))
    return annotations

def polygon_area(points):
    n = len(points)
    if n < 3:
        return 0
    area = 0.0
    for i in range(n):
        x1, y1 = points[i]
        x2, y2 = points[(i + 1) % n]
        area += x1 * y2
        area -= x2 * y1
    return abs(area) / 2.0

def categorize_area(total_area, affected_area):
    if total_area == 0:
        return "Total area cannot be zero."
    percentage = (affected_area / total_area) * 100
    if percentage < 0:
        return "Invalid affected area."
    elif percentage <= 4:
        category = "Normal"
    elif percentage <= 20:
        category = "Infected"
    elif percentage <= 40:
        category = "Serious"
    elif percentage <= 60:
        category = "Critical"
    elif percentage <= 80:
        category = "Very Critical"
    else:
        category = "Dead"
    return percentage, category

@app.route('/')
def index():
    return '''
        <h1>Upload Image for YOLOv8 Segmentation and Post-Processing</h1>
        <form action="/process" method="post" enctype="multipart/form-data">
            <label for="folder_name">Enter Folder Name:</label>
            <input type="text" id="folder_name" name="folder_name" required>
            <br><br>
            <input type="file" name="image" accept="image/*" required>
            <br><br>
            <input type="submit" value="Upload and Process">
        </form>
    '''

@app.route('/process', methods=['POST'])
def process():
    folder_name = request.form.get('folder_name')
    if 'image' not in request.files or not folder_name:
        return jsonify({"error": "Folder name or file part is missing"}), 400

    file = request.files['image']
    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    # Create user-specific folder for predictions
    output_folder = os.path.join(OUTPUT_FOLDER_BASE, folder_name)

    # Save uploaded image
    image_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(image_path)

    try:
        # YOLO command
        yolo_command = (
            f"yolo task=segment mode=predict model=/content/drive/MyDrive/30_epochs_pt22/weights/best.pt "
            f"conf=0.30 source={image_path} save_txt=True boxes=True project={OUTPUT_FOLDER_BASE} name={folder_name}"
        )
        subprocess.run(yolo_command, shell=True, check=True)

        # Post-process the uploaded image
        output_path = os.path.join(UPLOAD_FOLDER, f'processed_{file.filename}')
        processed_image = bluish_grey_to_black(image_path, output_path)
        leaf_area = calculate_leaf_area(processed_image)

        # Path to the folder containing segmented images with labels
        txt_dir = os.path.join('/content/drive/MyDrive/internship/predict/', folder_name, 'labels')  # Adjust according to the output
        total_area = 0.0

        for filename in os.listdir(txt_dir):
            if filename.endswith('.txt'):
                file_path = os.path.join(txt_dir, filename)
                annotations = read_yolo_instance_segmentation_file(file_path)
                for class_id, points in annotations:
                    area = polygon_area(points)
                    total_area += area

        percentage, category = categorize_area(leaf_area, total_area)

        # Prepare output images for display
        output_images = os.listdir(output_folder)

        # Filter only image files
        output_image_urls = [
            f"/predict/{folder_name}/{img}" for img in output_images
            if img.endswith(('.JPG', '.png', '.jpeg'))
        ]

        # HTML Response for displaying input and output images
        response_html = f'''
            <div style="background-color: black; padding: 20px; color: white; text-align: center;">
                <h2>Prediction complete!</h2>
                <div style="display: flex; justify-content: center; align-items: center;">
                    <div style="margin-right: 20px;">
                        <h3>Input Image</h3>
                        <img src="/uploads/{file.filename}" width="400" style="border: 2px solid white;">
                    </div>
                    <div>
                        <h3>Predicted Output</h3><br>

        '''
        if output_image_urls:
            for img_url in output_image_urls:
                response_html += f'<img src="{img_url}" width="400" style="border: 2px solid white;">'
        else:
            response_html += '<p>No output images found.</p>'

        response_html += f'''
    <div style="color: white; text-align: center; margin-top: 20px;">
        <h3>Leaf Area: {leaf_area} pixels</h3>
        <h3>Total Affected Area: {total_area:.2f} pixels</h3>
        <h3>Percentage: {percentage:.2f}%</h3>
        <h3>Category: {category}</h3>
    </div>
'''


        # Return the HTML response
        return render_template_string(response_html)

    # Exception handling
    except subprocess.CalledProcessError as e:
        return jsonify({"error": str(e)}), 500

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Serve input and processed images
@app.route('/uploads/<path:filename>')
def serve_image(filename):
    return send_from_directory(UPLOAD_FOLDER, filename)


@app.route('/predict/<path:folder_name>/<path:filename>')
def serve_output_image(folder_name, filename):
    folder_path = os.path.join(OUTPUT_FOLDER_BASE, folder_name)
    return send_from_directory(folder_path, filename)

if __name__ == "__main__":
    # Expose the Flask app on port 5000
    public_url = ngrok.connect(5000)
    print(f"Your app is running at: {public_url}")

    # Start the Flask app
    app.run(port=5000, debug=False)


Your app is running at: NgrokTunnel: "https://aeed-35-186-174-7.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 09:27:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 09:27:52] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 09:28:11] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 09:28:11] "GET /uploads/anushka.JPG HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 09:28:11] "GET /predict/folyd/anushka.JPG HTTP/1.1" 200 -
